In [1]:
from src.core_pro import AWS
from pathlib import Path

In [2]:
# bucket_name = 'sg-vnbi-proj-img-slide-api'
# s3 = AWS(bucket_name)

In [3]:
bucket_name = 'sg-vnbi-mkt-kevin'
s3 = AWS(bucket_name)
print(s3.get_all_files())

path = Path.home() / 'PycharmProjects/core_pro/how-to-use'
s3_key = 'voucher_sensitivity/prediction/2024-09-01_111_114'
# s3.download_file(path=path, key=s3_key)

🐸 S3: [Get] All files in: sg-vnbi-mkt-kevin
['voucher_sensitivity/prediction/2024-09-01_111_114', 'voucher_sensitivity/prediction/2024-09-01_17_114', 'voucher_sensitivity/prediction/2024-09-01_30_114', 'voucher_sensitivity/prediction/2024-09-01_39_114', 'voucher_sensitivity/prediction/2024-09-01_40_114', 'voucher_sensitivity/prediction/2024-09-01_49_114', 'voucher_sensitivity/prediction/2024-09-01_67_114', 'voucher_sensitivity/prediction/2024-09-01_72_114', 'voucher_sensitivity/prediction/2024-09-01_93_114', 'voucher_sensitivity/prediction/2024-09-01_9_114']


In [4]:
import polars as pl

df = pl.read_parquet(s3_key.split('/')[-1])
df.head()

create_datetime,userid,base_checkout_id,shop_id,seller_c2c,date_type,item_name,buyer_shipping_address_state,seller_shipping_address_state,route,cogs_range,order_cluster_global,order_main_cat_global,order_weight,shipping_channel,loyalty_tier,buyer_default_region_lvl1,buyer_lifecycle,fss_status,fsv_mbs,fsv_cap,fsv_voucher_discount,fsv_voucher_value,pv_mbs,pv_cap,pv_voucher_discount,pv_voucher_value,estimate_shipping_fee,buyer_paid_shipping_fee,merchandise_subtotal_amt,charged_weight,final_weight,scheme
datetime[μs],i32,i64,u32,cat,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i32,i32,i8,i32,i32,i32,i8,i32,f32,f32,f32,f64,f32,str
2024-09-01 22:10:09,274061729,178902609236639,69169839,"""Local_C2C""","""Mini Spike 1""","""Tấm hắt sáng chụp ảnh 60cm 2 m…","""TP. Hồ Chí Minh""","""TP. Hồ Chí Minh""","""Intra City""","""[0K, 45K)""","""ELHA""","""Cameras & Drones""","""[0kg - 1kg]""","""Standard""","""gold""","""Urban""","""Existing""","""FSS""",0,0,100,15000,null,null,null,null,16500.0,1500.0,32800.0,0.0,0.2,"""original"""
2024-09-01 14:43:47,874421678,178875827238075,2630589,"""Local_C2C""","""Mini Spike 1""","""Ô dù che mưa nắng chống tia uv…","""Nam Định""","""Hà Nội""","""Intra Region""","""[45K, 100K)""","""Fashion""","""Sports & Outdoors""","""[0kg - 1kg]""","""Standard""","""classic""","""Rural""","""Existing""","""FSS""",0,0,100,15000,null,null,null,null,32800.0,18000.0,48000.0,0.0,0.25,"""original"""
2024-09-01 20:57:46,1259940656,178898266241838,1256368397,"""Local_C2C""","""Mini Spike 1""","""Combo sơ mi nhựa Nylon + Vòng …","""An Giang""","""TP. Hồ Chí Minh""","""Intra Region""","""[45K, 100K)""","""ELHA""","""Home Appliances""","""[0kg - 1kg]""","""Standard""","""silver""","""Rural""","""Existing""","""FSS""",50000,0,100,25000,null,null,null,null,32800.0,8000.0,90000.0,0.02,0.11,"""original"""
2024-09-01 11:47:06,725329549,178865226208754,657326864,"""Local_C2C""","""Mini Spike 1""","""Bộ Combo Cần Câu Tay Đế Vương …","""Bình Dương""","""Hà Nội""","""Cross Region""","""[100K, 300K)""","""Fashion""","""Sports & Outdoors""","""[0kg - 1kg]""","""Standard""","""classic""","""Rural""","""New""","""FSS""",100000,0,100,300000,0,0,0,50000,50100.0,0.0,199000.0,0.475,0.51,"""original"""
2024-09-01 13:41:52,1186832765,178872112233853,894021566,"""Local_C2C""","""Mini Spike 1""","""Ba lô học sinh Túi đeo vai kép…","""Lai Châu""","""Hà Nội""","""Intra Region""","""[45K, 100K)""","""Fashion""","""Women Bags""","""[0kg - 1kg]""","""Standard""","""silver""","""Rural""","""Existing""","""FSS""",50000,0,100,15000,null,null,null,null,37600.0,23000.0,106000.0,0.501667,0.501667,"""original"""


In [6]:
df.schema

Schema([('create_datetime', Datetime(time_unit='us', time_zone=None)),
        ('userid', Int32),
        ('base_checkout_id', Int64),
        ('shop_id', UInt32),
        ('seller_c2c', Categorical(ordering='physical')),
        ('date_type', String),
        ('item_name', String),
        ('buyer_shipping_address_state', String),
        ('seller_shipping_address_state', String),
        ('route', String),
        ('cogs_range', String),
        ('order_cluster_global', String),
        ('order_main_cat_global', String),
        ('order_weight', String),
        ('shipping_channel', String),
        ('loyalty_tier', String),
        ('buyer_default_region_lvl1', String),
        ('buyer_lifecycle', String),
        ('fss_status', String),
        ('fsv_mbs', Int32),
        ('fsv_cap', Int32),
        ('fsv_voucher_discount', Int8),
        ('fsv_voucher_value', Int32),
        ('pv_mbs', Int32),
        ('pv_cap', Int32),
        ('pv_voucher_discount', Int8),
        ('pv_voucher_v

In [5]:
s3.convert_dataframe_flink_config(
    df,
    key=s3_key,
    table_name='dev_vnbi_mkt.ds_s3_voucher_sensitivity_fsv_only',
    primary_key='userid'
)


        [FLINK CONFIG]
        source {
            S3FileSource {
                datasource_name = "my_s3"
                path = "s3a://sg-vnbi-mkt-kevin/voucher_sensitivity/prediction/2024-09-01_111_114"
                format = "parquet"
                schema = '[{"field": "create_datetime", "type": null}, {"field": "userid", "type": "int"}, {"field": "base_checkout_id", "type": "bigint"}, {"field": "shop_id", "type": "int"}, {"field": "seller_c2c", "type": null}, {"field": "date_type", "type": "string"}, {"field": "item_name", "type": "string"}, {"field": "buyer_shipping_address_state", "type": "string"}, {"field": "seller_shipping_address_state", "type": "string"}, {"field": "route", "type": "string"}, {"field": "cogs_range", "type": "string"}, {"field": "order_cluster_global", "type": "string"}, {"field": "order_main_cat_global", "type": "string"}, {"field": "order_weight", "type": "string"}, {"field": "shipping_channel", "type": "string"}, {"field": "loyalty_tier", "type": "

In [ ]:
"""
env {
}

source {
    S3FileSource {
      datasource_name = "my_s3"
      path = "s3a://sg-vnbi-mkt-kevin/voucher_sensitivity/prediction/2024-09-01_111_114"
      format = "parquet"
      schema = 'b'[{"field":"create_datetime","type":null},{"field":"userid","type":"int"},{"field":"base_checkout_id","type":"bigint"},{"field":"shop_id","type":"int"},{"field":"seller_c2c","type":null},{"field":"date_type","type":"string"},{"field":"item_name","type":"string"},{"field":"buyer_shipping_address_state","type":"string"},{"field":"seller_shipping_address_state","type":"string"},{"field":"route","type":"string"},{"field":"cogs_range","type":"string"},{"field":"order_cluster_global","type":"string"},{"field":"order_main_cat_global","type":"string"},{"field":"order_weight","type":"string"},{"field":"shipping_channel","type":"string"},{"field":"loyalty_tier","type":"string"},{"field":"buyer_default_region_lvl1","type":"string"},{"field":"buyer_lifecycle","type":"string"},{"field":"fss_status","type":"string"},{"field":"fsv_mbs","type":"int"},{"field":"fsv_cap","type":"int"},{"field":"fsv_voucher_discount","type":"int"},{"field":"fsv_voucher_value","type":"int"},{"field":"pv_mbs","type":"int"},{"field":"pv_cap","type":"int"},{"field":"pv_voucher_discount","type":"int"},{"field":"pv_voucher_value","type":"int"},{"field":"estimate_shipping_fee","type":"float"},{"field":"buyer_paid_shipping_fee","type":"float"},{"field":"merchandise_subtotal_amt","type":"float"},{"field":"charged_weight","type":"double"},{"field":"final_weight","type":"float"},{"field":"scheme","type":"string"}]''
      endpoint = "https://s3g.data-infra.shopee.io"
      bucket = "sg-vnbi-mkt-kevin"
      access_key = "xuankhang.do"
      secret_key = "yZEbpqPxgHw9"
      ignore-parse-errors = "false"
      skip-first-data-row = "true"
      result_table_name = "s3FileSource"
    }
}

transform {
  Sql {
    sql = "select *, '${bizTime}' as grass_date from `s3FileSource`"
    result_table_name = "transformed"
  }
}

sink {
  HiveSink {
    source_table_name = "transformed"
    result_table_name = "dev_vnbi_mkt.ds_s3_voucher_sensitivity_fsv_only"
  }
}
"""

In [6]:
path = Path().home() / 'Desktop'
print(path)

s3.download_file(path=path, key='prm_2023-08-24.parquet')

C:\Users\Kevin\Desktop


Download - prm_2023-08-24.parquet: 100%|██████████| 13.7M/13.7M [00:05<00:00, 2.64MB/s]

🐸 S3: Download file successfully


In [8]:
file_path = path / 'prm_2023-08-24.parquet'
s3.upload_file(file=file_path)

Upload - prm_2023-08-24.parquet: 100%|██████████| 13.7M/13.7M [00:04<00:00, 3.23MB/s]

🐸 S3: Upload file successfully
